In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('../input/stumbleupon/train.tsv',delimiter='\t')
df_train.head()

In [ ]:
# Lets check for imbalanced dataset
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
sns.countplot(df_train.label)

In [ ]:
df_test=pd.read_csv('../input/stumbleupon/test.tsv',delimiter='\t')
df_test

In [ ]:
urlid = df_train["urlid"]
urlid

In [ ]:
## converting json to data frame
import json

title=[]
body=[]
urlid=[]
label=[]

for i in range(7395):
    X=json.loads(df_train["boilerplate"][i])
    if 'title' in X.keys() and 'body' in X.keys():
        title.append(X.get('title'))
        body.append(X.get('body'))
        urlid.append(df_train['urlid'][i])
        label.append(df_train['label'][i])
    

dictt = {'urlid':urlid,'title':title,'body':body,'label':label}
new = pd.DataFrame.from_dict(dictt) 

new.dropna()


In [ ]:
title=[]
body=[]
urlid=[]
label=[]

for i in range(3171):
    X=json.loads(df_test["boilerplate"][i])
    if 'title' in X.keys() and 'body' in X.keys():
        title.append(X.get('title'))
        body.append(X.get('body'))
        urlid.append(df_test['urlid'][i])
        
    

dictt = {'urlid':urlid,'title':title,'body':body}
new_test = pd.DataFrame.from_dict(dictt) 

new_test

In [ ]:
  for i in range(7393):
    if new['body'][i] is  None :
            new.drop(index=i,inplace=True)
    elif len(new['body'][i]) < 20:
            new.drop(index=i,inplace=True)
  new.reset_index(inplace=True)         

In [ ]:
new.drop(columns=['index'],inplace=True)
new

In [ ]:
y=new['label']
y

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
data=new.iloc[:,1:3]
data.replace("[^a-zA-Z]"," ",regex=True,inplace=True)
data['title']=data['title'].str.lower()
data['body']=data['body'].str.lower()
data



In [ ]:
titlebodycombined_train=[]
titlebodycombined_test=[]
for row in range(0,len(data.index)):
    titlebodycombined_train.append(' '.join(str(x) for x in data.iloc[row,1:3]))


for row in range(0,len(new_test.index)):
    titlebodycombined_test.append(' '.join(str(x) for x in new_test.iloc[row,1:3]))
titlebodycombined_test[0] 




In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps=PorterStemmer()
corpus=[]
stops = set(stopwords.words("english"))

for i in range(len(titlebodycombined_train)):
    tokens=nltk.word_tokenize(titlebodycombined_train[i])
    #taken only words(not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    #stem words 
    stemmed_list=[ps.stem(word) for word in token_words]
    #remove stopwords
    meaningful_words = [w for w in stemmed_list if not w in stops]
    rejoined_words = ( " ".join(meaningful_words))
    corpus.append(rejoined_words)

corpus[0]

In [ ]:
corpus1=[]
for i in range(len(titlebodycombined_test)):
    tokens=nltk.word_tokenize(titlebodycombined_test[i])
    #taken only words(not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    #stem words 
    stemmed_list=[ps.stem(word) for word in token_words]
    #remove stopwords
    meaningful_words = [w for w in stemmed_list if not w in stops]
    rejoined_words = ( " ".join(meaningful_words))
    corpus1.append(rejoined_words)
corpus1[0]    

In [ ]:
##feature extraction

from sklearn.feature_extraction.text import TfidfVectorizer
vect_word = TfidfVectorizer(lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
vect_char = TfidfVectorizer( lowercase=True, analyzer='char',
                        stop_words= 'english',ngram_range=(3,6),dtype=np.float32)

In [ ]:
from scipy import sparse
tr_vect = vect_word.fit_transform(corpus)
ts_vect = vect_word.transform(corpus1)

tr_vect_char = vect_char.fit_transform(corpus)
ts_vect_char = vect_char.transform(corpus1)




In [ ]:
X = sparse.hstack([tr_vect, tr_vect_char])
x_test = sparse.hstack([ts_vect, ts_vect_char])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc


lr = LogisticRegression(C=2,class_weight = 'balanced')
lr.fit(X,y)
prd = lr.predict_proba(x_test)


In [ ]:
pred =  lr.predict(X)
print('\nConfusion matrix\n',confusion_matrix(y,pred))
print(classification_report(y,pred))

In [ ]:
#prediction
y_pred = lr.predict(x_test)

  
data1 = {'urlid':new_test['urlid'],'label':y_pred}
    
submission_df =pd.DataFrame(data1)
    
submission_df.to_csv('Submission.csv',index=False)